In [ ]:
import tkinter as tk
from tkinter import simpledialog
import time
from DBIN import DatabaseInfo
from DeleteFile import DeleteAllFile # 폴더 필요
from Capture import capture_video
from Pattern_Cnn_Model_Run2 import PATTTERN_CNN_MODEL # 플레이어 필요
from Car_Cnn_Model_Run2 import CAR_CNN_MODEL # 플레이어 필요
from WhatCards import whatPattern
from WhatCards import whatCar
from Point import PointCount
import poker_hand

cnxn, curosor = DatabaseInfo()

: 

In [13]:
curosor.execute(select_Connection_query, 'TestGame', 'MasterID')
user1 = curosor.fetchone()
curosor.execute(select_Connection_query, 'TestGame', 'SubMaster')
user2 = curosor.fetchone()
print(user1, user2)

if (user1 is None) :
    print("사용자1이 없음")
elif (user2 is None): # 게임 아이디가 있지만 연결된 사용자가 없을 때 실행
    print("사용자2 없음.")
else: # 게임 아이디가 있고 연결된 사용자가 있을 때 실행
    print("연결된 사용자가 있습니다.")

(0, 'MasterID', 'TestGame') None
사용자2 없음.


In [4]:
# 데이터 삽입 쿼리
User_insert_query = """
INSERT INTO USER_TB (id, name, win_count, lose_count)
VALUES (?, ?, ?, ?);
"""
Game_insert_query = """
INSERT INTO GAME_TB (id, start_point, ante, start_time)
VALUES (?, ?, ?, ?);
"""
Connection_insert_query = """
INSERT INTO CONNECTION_TB (USER_TB_id, GAME_TB_id)
VALUES (?, ?);
"""
Round_insert_query = """
INSERT INTO ROUND_TB (GAME_TB_id, round, user1_card1, user_card2, user2_card1, user2_card2, user1_betting1, user2_betting1, dealer_card1, dealer_card2, dealer_card3, user1_betting2, user2_betting2, dealer_card4, user1_betting3, user2_betting3, dealer_card5, user1_betting4, user2_betting4, user1_hands, user2_hands, winner)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""
# 데이터 찾기 쿼리
select_User_query = """
SELECT *
FROM USER_TB
WHERE id = ?
"""
select_Game_query="""
SELECT *
FROM GAME_TB
WHERE id = ?
"""
select_Connection_query="""
SELECT *
FROM CONNECTION_TB
WHERE GAME_TB_id = ? and USER_TB_id = ?
"""

Round_select_query = """
SELECT *
FROM RoundTable
WHERE GAME_TB_id = ?
"""
select_Round_query="""
SELECT User1Batting1, User2Batting1, User1Batting2, User2Batting2, User1Batting3, User2Batting3, User1Batting4, User2Batting4
FROM RoundTable
WHERE GameID = ?
"""

In [56]:
class MainProgram:
    def __init__(self):
        self.root = tk.Tk()

        self.upper_frame = tk.Frame(self.root)
        self.upper_txt = tk.Text(self.upper_frame, width=30, height=40, state=tk.DISABLED)
        self.scrollbar = tk.Scrollbar(self.upper_frame, command=self.upper_txt.yview)

        self.player_button = tk.Button(self.root, text="Program Start", command = self.get_player_info)
        #self.database_button = tk.Button(self.root, text="Database", command=self.select_database)

    # 환면 구성
    def GUI(self):
        self.root.geometry("500x700+750+200")
        self.upper_frame.pack()
        self.upper_txt.pack(side=tk.LEFT, fill=tk.Y)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.player_button.pack()
        #self.database_button.pack()

        self.root.mainloop()

    # 화면에 값 반환 함수
    def update_display(self, message):
        self.upper_txt.config(state=tk.NORMAL)
        self.upper_txt.insert(tk.END, message)
        self.upper_txt.config(state=tk.DISABLED)
        self.upper_txt.yview_moveto(1.0)
        self.root.update()

    # 유저 정보 함수
    def get_player_info(self):
        UserNames = []  # 사용자 이름
        UserID = []     # 사용자 ID
        UserNum = 1 # 최소 사용자 수
        userNum = simpledialog.askinteger("플레이어 수", "플레이어 수를 입력하세요(2):", minvalue=2, maxvalue=2) # 최대 사용자 수
        
        while True:
            # 사용자 아이디와 이름 입력
            id = simpledialog.askstring(f"플레이어 {UserNum} 아이디", f"플레이어 {UserNum} 아이디 입력해주세요.")
            name = simpledialog.askstring(f"플레이어 {UserNum} 이름", f"플레이어 {UserNum} 이름 입력해주세요.")
            
            curosor.execute(select_User_query, id)
            row = curosor.fetchone()

            if row is None: # 아이디가 없을 때 실행
                UserNames.append(name)
                UserID.append(id)

                self.update_display("신규 이용자 성공\n")
                self.update_display(f"플레이어{UserNum} 아이디: {id}\n")
                self.update_display(f"플레이어{UserNum} 이름: {name}\n")
                
                curosor.execute(User_insert_query, id, name, 0, 0)
                cnxn.commit()
                UserNum += 1
                if UserNum > userNum:
                        break

            else: # 아이디가 있을 때 실행
                if row[1] == name: # 이름이 일치
                    UserNames.append(name)
                    UserID.append(id)

                    self.update_display("기존 이용자 성공\n")
                    self.update_display(f"플레이어{UserNum} 아이디: {UserID[UserNum-1]}\n")
                    self.update_display(f"플레이어{UserNum} 이름: {UserNames[UserNum-1]}\n")
                    UserNum += 1
                    if UserNum > userNum:
                        break

                else: # 이름이 불일치
                    self.update_display(f"플레이어 {UserNum} 이름 불일치!\n")
                    continue
                
        self.update_display("\n")
        self.root.after(1000, self.GameTable, UserID)

    # 게임 테이블 함수
    def GameTable(self, UserID):
        GameID = simpledialog.askstring("게임 테이블 아이디 입력", "테이블 아이디 입력해주세요.")
        self.update_display(f"게임 아이디: {GameID}\n")

        curosor.execute(select_Game_query, GameID)
        row = curosor.fetchone()

        if row is None: # 게임 아이디가 없을 때 실행
            point = simpledialog.askinteger("게임 시작 포인트", "게임 시작 포인트 입력해주세요.(1 ~ 1000000)", minvalue=1, maxvalue=1000000)
            ante = simpledialog.askinteger("시작 필수 포인트", f"시작 필수 포인트 입력해주세요.(0 ~ {point})", minvalue=0, maxvalue=point)
            now = time.localtime()
            now_time = "%04d-%02d-%02d" % (now.tm_year, now.tm_mon, now.tm_mday)
            
            self.update_display(f"게임 시작 포인트: {point}\n")
            self.update_display(f"시작 필수 포인트: {ante}\n")
            self.update_display(f"{now_time}에 신규 게임 테이블 생성\n")
            self.update_display("\n")

            curosor.execute(Game_insert_query, GameID, point, ante, now_time)
            curosor.execute(Connection_insert_query, UserID[0], GameID)
            curosor.execute(Connection_insert_query, UserID[1], GameID)
            cnxn.commit()
            # 유저 카드 확인 단계
            self.root.after(1000, self.player_receive_cards, GameID, UserID)

        else: # 게임 아이디가 있을 때 실행
            curosor.execute(select_Connection_query, GameID, UserID[0])
            user1 = curosor.fetchone()
            curosor.execute(select_Connection_query, GameID, UserID[1])
            user2 = curosor.fetchone()
            print(user1, user2)

            if (user1 is None) and (user2 is None): # 게임 아이디가 있지만 연결된 사용자가 없을 때 실행
                self.update_display(f"이미 {GameID} 게임 테이블이 있습니다.\n")
                self.update_display("다시 정해주세요.\n")
                self.update_display("\n")
                # 게임 테이블 입력 단계
                self.root.after(1000, self.GameTable, UserID)
            else: # 게임 아이디가 있고 연결된 사용자가 있을 때 실행
                self.update_display(f"{GameID} 게임 테이블 연결\n")
                self.update_display("\n")
                # 유저 카드 확인 단계
                self.root.after(1000, self.player_receive_cards, GameID, UserID)

In [57]:
MainProgram().GUI()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\tjddb\anaconda3\envs\Natural_env\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "c:\Users\tjddb\anaconda3\envs\Natural_env\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "C:\Users\tjddb\AppData\Local\Temp\ipykernel_3008\2990953732.py", line 100, in GameTable
    curosor.execute(Connection_insert_query, UserID[0], GameID)
pyodbc.ProgrammingError: ('The SQL contains 3 parameter markers, but 2 parameters were supplied', 'HY000')


In [21]:
class MainProgram:
    def __init__(self):
        self.root = tk.Tk()

        self.upper_frame = tk.Frame(self.root)
        self.upper_txt = tk.Text(self.upper_frame, width=30, height=40, state=tk.DISABLED)
        self.scrollbar = tk.Scrollbar(self.upper_frame, command=self.upper_txt.yview)

        self.button_new = tk.Button(self.root, text="프로그램 시작", command = self.get_player_info)
        self.button_2new = tk.Button(self.root, text="Database", command=self.select_database)

    def select_database(self):
        self.root2 = tk.Tk()
        self.upper_frame2 = tk.Frame(self.root2)
        self.upper_txt1212 = tk.Text(self.upper_frame2, width=150, height=40)
        self.scrollbar = tk.Scrollbar(self.upper_frame2, command=self.upper_txt.yview)
        self.root2.geometry("500x500+210+210")

        self.upper_frame2.pack()
        self.upper_txt.pack(side=tk.LEFT, fill=tk.Y)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)


        while True:
            What_Database = simpledialog.askinteger("어떤 데이터베이스를 열까요?", "1: 유저 정보, 2: 게임 테이블 정보, 3: 게임 정보", minvalue=1, maxvalue=3)
            if What_Database ==1:
                What_userID = simpledialog.askstring("플레이어 아이디", "아이디를 입력해주세요.")
                curosor.execute(select_User_query, What_userID)
                row = curosor.fetchone()
                if row is None:   # 아이디가 없으면 실행
                    continue
                else:   # 아이디가 있으면 실행
                    self.upper_txt1212.insert(1.0, f"아이디: {row.UserID}\n 이름: {row.UserName}\n 승리수: {row.WinnerCount}\n 패배수: {row.LoseCount}\n")
                    self.upper_txt1212.pack(side=tk.LEFT, fill=tk.Y)
                    continue

            if What_Database ==2:
                What_GameID = simpledialog.askstring("게임 테이블", "게임 테이블을 입력해주세요.")
                curosor.execute(select_Game_query, What_GameID)
                row = curosor.fetchone()
                if row is None:   # 게임 테이블이 없으면 실행
                    continue
                else:   # 게임 테이블이 있으면 실행
                    self.upper_txt1212.insert(1.0, f"게임 테이블: {row[0]}\n")
                    self.upper_txt1212.insert(1.0, f"유저1 아이디: {row[1]}\n")
                    self.upper_txt1212.insert(1.0, f"유저2 아이디: {row[2]}\n")
                    self.upper_txt1212.insert(1.0, f"시작 포인트: {row[3]}\n")
                    self.upper_txt1212.insert(1.0, f"시작 제출 포인트: {row[4]}\n")
                    self.upper_txt1212.insert(1.0, f"시작 시간: {row[5]}\n")

                    self.upper_txt1212.pack(side=tk.LEFT, fill=tk.Y)
                    continue

            if What_Database ==3:
                What_GameID = simpledialog.askstring("게임 테이블", "게임 테이블을 입력해주세요.")
                curosor.execute(Round_select_query, What_GameID)
                row = curosor.fetchone()
                if row is None:   # 아이디가 없으면 실행
                    continue
                else:   # 아이디가 있으면 실행
                    while row:
                        self.upper_txt1212.insert(1.0, f"게임 테이블: {row[1]}, 라운드: {row[2]}\n")
                        self.upper_txt1212.insert(1.0, f"유저1 카드: {row[3]}, {row[4]}\n")
                        self.upper_txt1212.insert(1.0, f"유저2 카드: {row[5]}, {row[6]}\n")
                        self.upper_txt1212.insert(1.0, f"딜러 카드: {row[9]}, {row[10]}, {row[11]} {row[14]} {row[17]}\n")
                        self.upper_txt1212.insert(1.0, f"유저1 족보: {row[20]}\n")
                        self.upper_txt1212.insert(1.0, f"유저2 족보: {row[21]}\n")
                        self.upper_txt1212.insert(1.0, f"승리자: {row[22]}\n")
                        self.upper_txt1212.pack(side=tk.LEFT, fill=tk.Y)
                        row = curosor.fetchone()
                        
    def YOLOv5MODEL():
        print("YOLOv5 실행")
        !cd yolov5 && python detect.py --source ../capture1_1.jpg --weights ../TrainModel/best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp1 --exist-ok --line-thickness 10
        !cd yolov5 && python detect.py --source ../capture1_2.jpg --weights ../TrainModel/best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp2 --exist-ok --line-thickness 10
        !cd yolov5 && python detect.py --source ../capture2_1.jpg --weights ../TrainModel/best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp3 --exist-ok --line-thickness 10
        !cd yolov5 && python detect.py --source ../capture2_2.jpg --weights ../TrainModel/best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp4 --exist-ok --line-thickness 10
        print("YOLOv5 실행 완료")

    # 화면에 값 반환 함수
    def update_display(self, message):
        self.upper_txt.config(state=tk.NORMAL)
        self.upper_txt.insert(tk.END, message)
        self.upper_txt.config(state=tk.DISABLED)
        self.upper_txt.yview_moveto(1.0)
        self.root.update()

    # 환면 구성
    def GUI(self):
        self.root.geometry("500x700+750+200")
        self.upper_frame.pack()
        self.upper_txt.pack(side=tk.LEFT, fill=tk.Y)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.button_new.pack()
        self.button_2new.pack()

        self.root.mainloop()

    # 유저 정보 함수
    def get_player_info(self):
        Names = []  # 사용자 이름
        ID = []     # 사용자 ID
        UserNum = 1 # 사용자 번호

        userNum = simpledialog.askinteger("플레이어 수", "플레이어 수를 입력하세요(2):", minvalue=2, maxvalue=2)
        
        while True:
            # 사용자 아이디와 이름 입력
            userID = simpledialog.askstring(f"플레이어 {UserNum} 아이디", f"플레이어 {UserNum} 아이디 입력해주세요.")
            userName = simpledialog.askstring(f"플레이어 {UserNum} 이름", f"플레이어 {UserNum} 이름 입력해주세요.")
            
            # 아이디가 있는지 확인
            curosor.execute(select_User_query, userID)
            row = curosor.fetchone()
            if row is None:   # 아이디가 없으면 실행
                Names.append(userName)
                ID.append(userID)

                self.update_display("신규 이용자 성공\n")
                self.update_display(f"플레이어{UserNum} 아이디: {userID}\n")
                self.update_display(f"플레이어{UserNum} 이름: {userName}\n")

                curosor.execute(User_insert_query, userID, userName, 0, 0)
                cnxn.commit()
                UserNum += 1
                if UserNum > userNum:
                        break
            else:   # 아이디가 있으면 실행
                if row[1] == userName:
                    Names.append(userName)
                    ID.append(userID)

                    self.update_display("기존 이용자 성공\n")
                    self.update_display(f"플레이어{UserNum} 아이디: {ID[UserNum-1]}\n")
                    self.update_display(f"플레이어{UserNum} 이름: {Names[UserNum-1]}\n")
                    UserNum += 1
                    if UserNum > userNum:
                        break
                else:
                    self.update_display(f"플레이어 {UserNum} 이름 불일치!\n")
                    continue
                
        self.update_display("\n")
        self.root.after(1000, self.GameTable, ID)

    # 게임 테이블 함수
    def GameTable(self, ID):
        GameID = simpledialog.askstring("게임 테이블 아이디 입력", "테이블 아이디 입력해주세요.")
        self.update_display(f"게임 아이디: {GameID}\n")

        curosor.execute(select_Game_query, GameID)
        row = curosor.fetchone()
        if row is None:
            point = simpledialog.askinteger("게임 시작 포인트", "게임 시작 포인트 입력해주세요.(1 ~ 1000000)", minvalue=1, maxvalue=1000000)
            startingPoint = simpledialog.askinteger("시작 필수 포인트", f"시작 필수 포인트 입력해주세요.(0 ~ {point})", minvalue=0, maxvalue=point)
            now = time.localtime()
            now_time = "%04d-%02d-%02d" % (now.tm_year, now.tm_mon, now.tm_mday)
            
            self.update_display(f"게임 시작 포인트: {point}\n")
            self.update_display(f"시작 필수 포인트: {startingPoint}\n")
            self.update_display(f"{now_time}에 신규 게임 테이블 생성\n")
            self.update_display("\n")

            curosor.execute(Game_insert_query, GameID, point, startingPoint, now_time)
            cnxn.commit()
            # 유저 카드 확인 단계
            self.root.after(1000, self.player_receive_cards, GameID, ID)

        else:
            if row[1] == ID[0] and row[2] == ID[1]:
                self.update_display(f"{GameID} 게임 테이블 연결\n")
                self.update_display("\n")
                # 유저 카드 확인 단계
                self.root.after(1000, self.player_receive_cards, GameID, ID)
            else:
                self.update_display(f"이미 {GameID} 게임 테이블이 있습니다.\n")
                self.update_display("다시 정해주세요.\n")
                self.update_display("\n")
                # 게임 테이블 입력 단계
                self.root.after(1000, self.GameTable, ID)

    # 유저의 카드 정보 확인
    def player_receive_cards(self, GameID, ID):
        PlayerCardList = [0, 0, 0, 0, 0 , 0, 0, 0]

        for i in range(1, 5):
            i = str(i)
            DeleteAllFile("./runs/detect/exp" + i + "/crops/Character/")
            DeleteAllFile("./runs/detect/exp" + i + "/crops/Pattern")

        capture_video()
        !cd yolov5 && python detect.py --source ../capture1_1.jpg --weights ../TrainModel/Pattern_Car_best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp1 --exist-ok --line-thickness 10
        !cd yolov5 && python detect.py --source ../capture1_2.jpg --weights ../TrainModel/Pattern_Car_best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp2 --exist-ok --line-thickness 10
        !cd yolov5 && python detect.py --source ../capture2_1.jpg --weights ../TrainModel/Pattern_Car_best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp3 --exist-ok --line-thickness 10
        !cd yolov5 && python detect.py --source ../capture2_2.jpg --weights ../TrainModel/Pattern_Car_best.pt --img 640 --conf 0.3 --iou 0.4 --save-txt --save-crop --project ../runs/detect --name exp4 --exist-ok --line-thickness 10

        try:
            CardList = []
            for i in range(1, 3):
                play_Pattern = PATTTERN_CNN_MODEL(i)
                play_Car = CAR_CNN_MODEL(i)

                if play_Pattern == "Error" or play_Car == "Error":
                    raise
                
                CardList.append(play_Pattern)
                CardList.append(play_Car)

            PlayerCardList[0] = CardList[0]
            PlayerCardList[1] = CardList[1]
            PlayerCardList[2] = CardList[2]
            PlayerCardList[3] = CardList[3]
        except:
            self.update_display("유저1 카드 식별 실패\n")
            self.update_display("직접 입력해주세요.\n")
            play1_Pattern1 = simpledialog.askinteger(f"유저1 첫번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
            play1_Car1 = simpledialog.askinteger(f"유저1 첫번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
            play1_Pattern2 = simpledialog.askinteger(f"유저1 두번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
            play1_Car2 = simpledialog.askinteger(f"유저1 두번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
            
            play_Pattern1 = whatPattern(play1_Pattern1)
            play_Car1 = whatCar(play1_Car1)
            play_Pattern2 = whatPattern(play1_Pattern2)
            play_Car2 = whatCar(play1_Car2)

            PlayerCardList[0] = play_Pattern1
            PlayerCardList[1] = play_Car1
            PlayerCardList[2] = play_Pattern2
            PlayerCardList[3] = play_Car2

        try:
            CardList = []
            for i in range(3, 5):
                play_Pattern = PATTTERN_CNN_MODEL(i)
                play_Car = CAR_CNN_MODEL(i)
                
                if play_Pattern == "Error" or play_Car == "Error":
                    raise
                
                CardList.append(play_Pattern)
                CardList.append(play_Car)
            
            PlayerCardList[4] = CardList[0]
            PlayerCardList[5] = CardList[1]
            PlayerCardList[6] = CardList[2]
            PlayerCardList[7] = CardList[3]
        except:
            self.update_display("유저2 카드 식별 실패\n")
            self.update_display("직접 입력해주세요.\n")
            play2_Pattern1 = simpledialog.askinteger(f"유저2 첫번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
            play2_Car1 = simpledialog.askinteger(f"유저2 첫번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
            play2_Pattern2 = simpledialog.askinteger(f"유저2 두번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
            play2_Car2 = simpledialog.askinteger(f"유저2 두번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
            
            play_Pattern1 = whatPattern(play2_Pattern1)
            play_Car1 = whatCar(play2_Car1)
            play_Pattern2 = whatPattern(play2_Pattern2)
            play_Car2 = whatCar(play2_Car2)

            PlayerCardList[4] = play_Pattern1
            PlayerCardList[5] = play_Car1
            PlayerCardList[6] = play_Pattern2
            PlayerCardList[7] = play_Car2

        self.update_display(f"유저1 첫번째 카드: {PlayerCardList[0]}, {PlayerCardList[1]}\n")
        self.update_display(f"유저1 두번째 카드: {PlayerCardList[2]}, {PlayerCardList[3]}\n")
        self.update_display(f"유저2 첫번째 카드: {PlayerCardList[4]}, {PlayerCardList[5]}\n")
        self.update_display(f"유저2 두번째 카드: {PlayerCardList[6]}, {PlayerCardList[7]}\n")
        self.update_display("\n")

        # 배팅 단계
        self.root.after(1000, self.Gameing, GameID, ID, PlayerCardList)
    
    # 배팅 단계
    def Gameing(self,GameID, ID, PlayerCardList):
        User1Point, User2Point, Count = PointCount(GameID)
        Batting = [0, 0, 0, 0, 0, 0, 0, 0]
        DealerCard = [None, None, None, None, None, None,
                      None, None,
                      None, None]

        try:
            play1_Betting = simpledialog.askinteger("유저1 첫번째 배팅 포인트 입력해주세요.", f"최대: {User1Point}", minvalue=0, maxvalue=User1Point)
            play2_Betting = simpledialog.askinteger("유저2 첫번째 배팅 포인트를 입력해주세요.", f"최대: {User2Point}", minvalue=0, maxvalue=User2Point)
            self.update_display(f"유저1 첫번째 배팅 포인트: {play1_Betting}\n")
            self.update_display(f"유저2 첫번째 배팅 포인트: {play2_Betting}\n")
            self.update_display("\n")

            User1Point -= play1_Betting
            User2Point -= play2_Betting
            Batting[0] = play1_Betting
            Batting[1] = play2_Betting
            if (play1_Betting != play2_Betting):
                if (play1_Betting > play2_Betting):
                    Winner = ID[0]
                else:
                    Winner = ID[1]
                raise
            else:
                Dealer1_Card1_Pattern = simpledialog.askinteger("딜러 첫번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
                Dealer1_Card1_Car = simpledialog.askinteger("딜러 첫번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
                Dealer1_Card2_Pattern = simpledialog.askinteger("딜러 두번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
                Dealer1_Card2_Car = simpledialog.askinteger("딜러 두번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
                Dealer1_Card3_Pattern = simpledialog.askinteger("딜러 세번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
                Dealer1_Card3_Car = simpledialog.askinteger("딜러 세번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
                
                DealerCard_Pattern = whatPattern(Dealer1_Card1_Pattern)
                DealerCard_Car = whatCar(Dealer1_Card1_Car)
                DealerCard[0] = DealerCard_Pattern
                DealerCard[1] = DealerCard_Car

                DealerCard_Pattern = whatPattern(Dealer1_Card2_Pattern)
                DealerCard_Car = whatCar(Dealer1_Card2_Car)
                DealerCard[2] = DealerCard_Pattern
                DealerCard[3] = DealerCard_Car
    
                DealerCard_Pattern = whatPattern(Dealer1_Card3_Pattern)
                DealerCard_Car = whatCar(Dealer1_Card3_Car)
                DealerCard[4] = DealerCard_Pattern
                DealerCard[5] = DealerCard_Car

                self.update_display(f"딜러 첫번째 카드: {DealerCard[0]}, {DealerCard[1]}\n")
                self.update_display(f"딜러 두번째 카드: {DealerCard[2]}, {DealerCard[3]}\n")
                self.update_display(f"딜러 세번째 카드: {DealerCard[4]}, {DealerCard[5]}\n")
                self.update_display("\n")
                
                play1_Betting = simpledialog.askinteger("유저1 두번째 배팅 포인트 입력해주세요.", f"최대: {User1Point}", minvalue=0, maxvalue=User1Point)
                play2_Betting = simpledialog.askinteger("유저2 두번째 배팅 포인트를 입력해주세요.", f"최대: {User2Point}", minvalue=0, maxvalue=User2Point)
                self.update_display(f"유저1 두번째 배팅 포인트: {play1_Betting}\n")
                self.update_display(f"유저2 두번째 배팅 포인트: {play2_Betting}\n")
                self.update_display("\n")
                
                User1Point -= play1_Betting
                User2Point -= play2_Betting
                Batting[2] = play1_Betting
                Batting[3] = play2_Betting

                if (play1_Betting != play2_Betting):
                    if (play1_Betting > play2_Betting):
                        Winner = ID[0]
                    else:
                        Winner = ID[1]
                    raise
                else:
                    Dealer1_Card4_Pattern = simpledialog.askinteger(f"딜러 네번째 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
                    Dealer1_Card4_Car = simpledialog.askinteger(f"딜러 네번째 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
                    
                    DealerCard_Pattern = whatPattern(Dealer1_Card4_Pattern)
                    DealerCard_Car = whatCar(Dealer1_Card4_Car)
                    DealerCard[6] = DealerCard_Pattern
                    DealerCard[7] = DealerCard_Car

                    self.update_display(f"딜러 네번째 카드: {DealerCard[6]}, {DealerCard[7]}\n")
                    self.update_display("\n")
                    
                    play1_Betting = simpledialog.askinteger("유저1 세번째 배팅 포인트 입력해주세요.", f"최대: {User1Point}", minvalue=0, maxvalue=User1Point)
                    play2_Betting = simpledialog.askinteger("유저2 세번째 배팅 포인트를 입력해주세요.", f"최대: {User2Point}", minvalue=0, maxvalue=User2Point)
                    self.update_display(f"유저1 세번째 배팅 포인트: {play1_Betting}\n")
                    self.update_display(f"유저2 세번째 배팅 포인트: {play2_Betting}\n")
                    self.update_display("\n")
                    
                    Batting[4] = play1_Betting
                    Batting[5] = play2_Betting

                    if (play1_Betting != play2_Betting):
                        if (play1_Betting > play2_Betting):
                            Winner = ID[0]
                        else:
                            Winner = ID[1]
                        raise
                    else:
                        Dealer1_Card5_Pattern = simpledialog.askinteger("딜러 마지막 카드 문양을 입력해주세요.", "0:Clover, 1:Diamond, 2:Heart, 3:Spade", minvalue=0, maxvalue=3)
                        Dealer1_Card5_Car = simpledialog.askinteger("딜러 마지막 카드 문자를 입력해주세요.", "0:A, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7, 7:8, 8:9, 9:10, 10:J, 11:Q, 12:K", minvalue=0, maxvalue=12)
                        
                        DealerCard_Pattern = whatPattern(Dealer1_Card5_Pattern)
                        DealerCard_Car = whatCar(Dealer1_Card5_Car)
                        DealerCard[8] = DealerCard_Pattern
                        DealerCard[9] = DealerCard_Car

                        self.update_display(f"딜러 네번째 카드: {DealerCard[8]}, {DealerCard[9]}\n")
                        self.update_display("\n")
                        
                        play1_Betting = simpledialog.askinteger("유저1 마지막 배팅 포인트 입력해주세요.", f"최대: {User1Point}", minvalue=0, maxvalue=User1Point)
                        play2_Betting = simpledialog.askinteger("유저2 마지막 배팅 포인트를 입력해주세요.", f"최대: {User2Point}", minvalue=0, maxvalue=User2Point)
                        self.update_display(f"유저1 마지막 배팅 포인트: {play1_Betting}\n")
                        self.update_display(f"유저2 마지막 배팅 포인트: {play2_Betting}\n")
                        self.update_display("\n")
                        
                        Batting[6] = play1_Betting
                        Batting[7] = play2_Betting

                        if (play1_Betting != play2_Betting):
                            if (play1_Betting > play2_Betting):
                                Winner = ID[0]
                            else:
                                Winner = ID[1]
                            raise
                        else:
                            raise
        except:
            p1 = poker_hand.Player(ID[0])
            p1.get_cards(poker_hand.Card(PlayerCardList[0], PlayerCardList[1]))
            p1.get_cards(poker_hand.Card(PlayerCardList[2], PlayerCardList[3]))

            p2 = poker_hand.Player(ID[1])
            p2.get_cards(poker_hand.Card(PlayerCardList[4], PlayerCardList[5]))
            p2.get_cards(poker_hand.Card(PlayerCardList[6], PlayerCardList[7]))

            Dealer = poker_hand.Card(DealerCard[0], DealerCard[1]), poker_hand.Card(DealerCard[2], DealerCard[3]), poker_hand.Card(DealerCard[4], DealerCard[5]), poker_hand.Card(DealerCard[6], DealerCard[7]), poker_hand.Card(DealerCard[8], DealerCard[9])
            p1.get_dealer_cards(Dealer)
            p2.get_dealer_cards(Dealer)
            Winner = poker_hand.win_lose(p1, p2)

            if Winner == "draw":
                self.update_display("무승부\n")
            else:
                self.update_display(f"승리자: {Winner}\n")

            curosor.execute(Round_insert_query, GameID, Count,
                            f"{PlayerCardList[0]} {PlayerCardList[1]}", f"{PlayerCardList[2]} {PlayerCardList[3]}", f"{PlayerCardList[4]} {PlayerCardList[5]}", f"{PlayerCardList[6]} {PlayerCardList[7]}",
                            Batting[0], Batting[1], f"{DealerCard[0]} {DealerCard[1]}", f"{DealerCard[2]} {DealerCard[3]}", f"{DealerCard[4]} {DealerCard[5]}",
                            Batting[2], Batting[3], f"{DealerCard[6]} {DealerCard[7]}",
                            Batting[4], Batting[5], f"{DealerCard[8]} {DealerCard[9]}",
                            Batting[6], Batting[7], poker_hand.get_rank(p1).name, poker_hand.get_rank(p2).name, Winner)
            cnxn.commit()

In [22]:
MainProgram().GUI()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\tjddb\anaconda3\envs\Natural_env\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "c:\Users\tjddb\anaconda3\envs\Natural_env\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "C:\Users\tjddb\AppData\Local\Temp\ipykernel_3008\2064270769.py", line 165, in GameTable
    curosor.execute(Game_insert_query, GameID, ID[0], ID[1], point, startingPoint, now_time)
pyodbc.ProgrammingError: ('The SQL contains 4 parameter markers, but 6 parameters were supplied', 'HY000')


In [18]:
import poker_hand

p1 = poker_hand.Player("a")
p2 = poker_hand.Player("b")
p1.get_cards(poker_hand.Card("Heart", "10"))
p1.get_cards(poker_hand.Card("Clover", "5"))

p2.get_cards(poker_hand.Card("Diamond", "4"))
p2.get_cards(poker_hand.Card("Spade", "A"))
print(p1.cards[0].pattern, p1.cards[0].character)
print(p1.cards[1].pattern, p1.cards[1].character)
print(p2.cards[0].pattern, p2.cards[0].character)
print(p2.cards[1].pattern, p2.cards[1].character)

Dealer = poker_hand.Card("Diamond", "J"), poker_hand.Card("Clover", "2"), poker_hand.Card("Clover", "J"), poker_hand.Card("Spade", "6"), poker_hand.Card("Diamond", "8")
p1.get_dealer_cards(Dealer)
p2.get_dealer_cards(Dealer)

print(poker_hand.win_lose(p1, p2))
print(poker_hand.get_rank(p1).name)
print(poker_hand.get_rank(p2).name)
print(Dealer[0].pattern, Dealer[0].character)


Heart 10
Clover 5
Diamond 4
Spade A
2
draw
One_Pair
One_Pair
Diamond J
